In [1]:
import pandas as pd

In [2]:
def firstnum(info_item):
    """
    Get the index of first number in cpt table
    """
    for i,string in enumerate(info_item):
        try:
            if float(string):
                break
        except:
            pass
        
    return i

In [3]:
def file2dict(Filename):
    """
    Read file and transfer to a dictionary which saves all information
    """
    # load data
    data = pd.read_csv(Filename,header = None,delimiter = '\t')
    # split data and save it to a list
    info_list=[] 
    for i in data.index:
        info_list.append(data.iloc[i,0].split())
    # arrange the information and save it to a list
    info_dict={}
    for i in info_list:
        if 'END' in i:
            break
        else:
            child = i[0]
            parents = i[1:firstnum(i)]
            cpt = i[firstnum(i):]
            info_dict[child]={'parents':parents,'cpt':cpt}
    return info_dict

In [4]:
def construct(info_dict):
    """
    Construct the all possible full joint distributions and 
    save it as a list named result
    """
    temp = []
    result = []
    for i in range(2**len(info_dict)):
        binary = "{0:b}".format(i)
        binary = '0'*(len(info_dict)-len(binary))+binary
        temp.append(binary)
    
    for label in temp:
        fjd = []
        for j,node in enumerate(info_dict):
            if label[j]=='1':
                fjd.append('n'+node)
            else:
                fjd.append(node)
        result.append(fjd)
    return result

In [12]:
def p_fjd(fjd,info_dict):
    """
    This function is for the computation of each full joint distribution
    1.probability
    2.number of multiplication
    3.number of add
    """
    count_mul = 0
    count_add = 0
    probs = 1
    for i in fjd:
    # for situations without n in front of a node
        if i[0]!='n':
            if 'NONE' in info_dict[i]['parents']:
                prob = float(info_dict[i]['cpt'][0])
            else:
                # find the probability index given all parents
                binary = ''
                for parent in info_dict[i]['parents']:
                    if parent in fjd:
                        binary +='0'
                    else:
                        binary+='1'
                decimal = int(binary,2)
                prob = float(info_dict[i]['cpt'][decimal])
        else:
            i = i[1:]
            if 'NONE' in info_dict[i]['parents']:
                prob = 1.0-float(info_dict[i]['cpt'][0])
            else:
                binary = ''
                for parent in info_dict[i]['parents']:
                    if parent in fjd:
                        binary +='0'
                    else:
                        binary+='1'
                decimal = int(binary,2)
                prob = 1.0- float(info_dict[i]['cpt'][decimal])
            count_add+=1
        probs *=prob
        count_mul+=1
    return probs,count_mul,count_add

In [90]:
def output(Filename):
    info_dict = file2dict(Filename)
    fjd_list = construct(info_dict)
    
    num_fjd = 2**len(info_dict)
    num_cpt = 0
    num_add = 0
    for node in info_dict:
        num_cpt += len(info_dict[node]['cpt'])
    compactness = float(num_cpt)/float(num_fjd)
    
    for i in fjd_list:
   
        p, count_multi, count_add = p_fjd(i,info_dict)

        text_file = open("Output.txt", "a")
        text_file.write(str(fjd)+'\t'+str(p)+'\n')

    
        num_add += count_add
    text_file.close()
    
    num_multi = len(fjd_list)*count_multi
    print '------------------------------------------------------'
    print 'Bayesian Network                  :\t'+Filename
    print 'Num of lines in joint distribution:\t',num_fjd
    print 'Num of CPT lines                  :\t',num_cpt
    print 'Compactness                       :\t',compactness
    print 'Num multiply operations           :\t',num_multi
    print 'Num add operations                :\t',num_add
    print '------------------------------------------------------'

In [91]:
output('C:/Users/youch/Desktop/final_AI/example.txt')

------------------------------------------------------
Bayesian Network                  :	C:/Users/youch/Desktop/final_AI/example.txt
Num of lines in joint distribution:	32
Num of CPT lines                  :	10
Compactness                       :	0.3125
Num multiply operations           :	160
Num add operations                :	80
------------------------------------------------------
